In [4]:

import math
import csv
import random
import numpy as np

# --- Ground truth parameters ---
S0 = 1.0      # Baseline signal
f = 0.3       # Perfusion fraction
D = 0.0010    # Diffusion coefficient (mm^2/s)
Dstar = 0.010 # Pseudo-diffusion coefficient (mm^2/s)

# --- b-values ---

b_full = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 150, 250, 400, 800, 1000]
b_whitepaper = [0, 30, 70, 100, 200, 400, 800]
b_subset_low = [0, 30, 70, 100]
b_subset_high = [0, 200, 400, 800]
b_minimal = [0, 100, 800]

# --- Compute S(b) ---
def compute_signals(b_values, S0, f, D, Dstar):
    return [S0 * (f * math.exp(-b * (D + Dstar)) + (1 - f) * math.exp(-b * D)) for b in b_values]

# --- Write CSV with metadata ---
def write_csv(filename, b_values, signals, S0, f, D, Dstar):
    with open(filename, "w", newline="") as f_out:
        writer = csv.writer(f_out)
        writer.writerow(["# IVIM two-compartment model: S(b) = S0 * [ f*exp(-b*(D+D*)) + (1-f)*exp(-b*D) ]"])
        writer.writerow([f"# Parameters: S0={S0}, f={f}, D={D} mm^2/s, D*={Dstar} mm^2/s"])
        writer.writerow(["# Units: b [s/mm^2], S [a.u.]"])
        writer.writerow(["b", "S"])
        for b, S in zip(b_values, signals):
            writer.writerow([b, f"{S:.6f}"])
    print(f"CSV written: {filename}")

# --- Full set ---
signals_full = compute_signals(b_full, S0, f, D, Dstar)
write_csv("ivim_signal_full.csv", b_full, signals_full, S0, f, D, Dstar)

# --- Subsets ---
subsets = {
    "whitepaper": b_whitepaper,
    "low": b_subset_low,
    "high": b_subset_high,
    "minimal": b_minimal
}
for name, bvals in subsets.items():
    signals = compute_signals(bvals, S0, f, D, Dstar)
    write_csv(f"ivim_signal_{name}.csv", bvals, signals, S0, f, D, Dstar)

# --- Uniform random noise ---
noise_signals_uniform = [random.uniform(0.0, 1.0) for _ in b_full]
write_csv("ivim_signal_noise_uniform.csv", b_full, noise_signals_uniform, S0, f, D, Dstar)

# --- Gaussian noise with controllable SNR ---
def add_gaussian_noise(signals, snr):
    sigma = S0 / snr  # SNR = S0 / sigma
    noisy_signals = [max(0.0, s + np.random.normal(0, sigma)) for s in signals]
    return noisy_signals

for snr in [10, 20, 40]:
    noisy_signals = add_gaussian_noise(signals_full, snr)
    write_csv(f"ivim_signal_gaussian_SNR{snr}.csv", b_full, noisy_signals, S0, f, D, Dstar)

# --- Print summary ---
print("\nGround truth parameters:")
print(f"S0 = {S0}, f = {f}, D = {D} mm^2/s, D* = {Dstar} mm^2/s")
print("\nFull set signals:")
for b, S in zip(b_full, signals_full):
    print(f"b = {b:>4} s/mm^2 -> S(b) = {S:.6f}")

print("\nNoise signals (uniform):")
for b, S in zip(b_full, noise_signals_uniform):
    print(f"b = {b:>4} s/mm^2 -> S(noise) = {S:.6f}")

print("\nGaussian noise cases:")
for snr in [10, 20, 40]:
    noisy_signals = add_gaussian_noise(signals_full, snr)
    print(f"SNR={snr}: example noisy signals -> {[f'{val:.6f}' for val in noisy_signals]}")


CSV written: ivim_signal_full.csv
CSV written: ivim_signal_whitepaper.csv
CSV written: ivim_signal_low.csv
CSV written: ivim_signal_high.csv
CSV written: ivim_signal_minimal.csv
CSV written: ivim_signal_noise_uniform.csv
CSV written: ivim_signal_gaussian_SNR10.csv
CSV written: ivim_signal_gaussian_SNR20.csv
CSV written: ivim_signal_gaussian_SNR40.csv

Ground truth parameters:
S0 = 1.0, f = 0.3, D = 0.001 mm^2/s, D* = 0.01 mm^2/s

Full set signals:
b =    0 s/mm^2 -> S(b) = 1.000000
b =   20 s/mm^2 -> S(b) = 0.926895
b =   30 s/mm^2 -> S(b) = 0.894989
b =   40 s/mm^2 -> S(b) = 0.865764
b =   50 s/mm^2 -> S(b) = 0.838946
b =   60 s/mm^2 -> S(b) = 0.814291
b =   70 s/mm^2 -> S(b) = 0.791580
b =   80 s/mm^2 -> S(b) = 0.770616
b =   90 s/mm^2 -> S(b) = 0.751225
b =  100 s/mm^2 -> S(b) = 0.733248
b =  120 s/mm^2 -> S(b) = 0.700985
b =  150 s/mm^2 -> S(b) = 0.660111
b =  250 s/mm^2 -> S(b) = 0.564339
b =  400 s/mm^2 -> S(b) = 0.472907
b =  800 s/mm^2 -> S(b) = 0.314575
b = 1000 s/mm^2 -> S(b)